<a href="https://colab.research.google.com/github/amelrnt/Machine_Learning_Project/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-09-13 00:26:35--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  13.3MB/s    in 1.9s    

2021-09-13 00:26:37 (13.3 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [ ]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [ ]:
df_ratings = df_ratings[df_ratings.rating >= 1.0]
df_ratings = df_ratings.reset_index(drop = True)
df_ratings

,user,isbn,rating
0,276726,0155061224,5.0
1,276729,052165615X,3.0
2,276729,0521795028,6.0
3,276736,3257224281,8.0
4,276737,0600570967,6.0
...,...,...,...
433666,276704,0806917695,5.0
433667,276704,1563526298,9.0
433668,276709,0515107662,10.0
433669,276721,0590442449,10.0


In [ ]:
df_valid = df_ratings.merge(df_books, how='inner').drop(['author'], 1)
df_valid

,user,isbn,rating,title
0,276726,0155061224,5.0,Rites of Passage
1,276729,052165615X,3.0,Help!: Level 1
2,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...
3,276744,038550120X,7.0,A Painted House
4,11676,038550120X,10.0,A Painted House
...,...,...,...,...
383847,276688,0425156737,2.0,Graven Images
383848,276688,0671563149,6.0,MUDDY WATER (Peter Bartholomew Mysteries)
383849,276688,1575660792,7.0,Gray Matter
383850,276704,0380796155,5.0,White Abacus


In [ ]:
plt.hist(df_ratings, 100)
plt.show()

In [ ]:
#create new datasets user with ratings count and >= 200 rates
df_ratings_count = df_valid.groupby('user').size().to_frame()
df_ratings_count.rename(columns = {0:'user_count'}, inplace = True)
df_ratings_count = df_ratings_count[df_ratings_count.user_count >= 200]
df_ratings_count = df_ratings_count.reset_index()
df_ratings_count.rename(columns = {'index':'user'}, inplace = True)
df_ratings_count

,user,user_count
0,4385,212
1,6251,217
2,6575,233
3,7346,314
4,11676,6943
...,...,...
113,265115,206
114,265889,227
115,269566,205
116,270713,257


In [ ]:
# clean rating data
df_ratings_new = df_ratings_count.merge(df_valid, how='left')
df_ratings_new = df_ratings_new.reset_index(drop=True)
df_ratings_new

,user,user_count,isbn,rating,title
0,4385,212,0312980353,10.0,Circle of Stars
1,4385,212,0373076746,10.0,Morning Side Of Dawn (Silhouette Intimate Mome...
2,4385,212,1551660016,10.0,The Devil's Own
3,4385,212,0373169299,10.0,At the Rancher's Bidding (Harlequin American R...
4,4385,212,0440221471,10.0,The Runaway Jury
...,...,...,...,...,...
58171,274061,211,1568821468,10.0,The Antarktos Cycle: Horror and Wonder at the ...
58172,274061,211,1569311803,10.0,No Need for Tenchi! (Book 1)
58173,274061,211,1569312540,10.0,Sword Play (No Need for Tenchi! Book 2)
58174,274061,211,1891153870,9.0,Book of All Flesh


In [ ]:
#create new datasets book with ratings count and >= 100 ratings
df_books_count = df_valid.groupby('isbn').size().to_frame()
df_books_count.rename(columns = {0:'book_count'}, inplace = True)
df_books_count = df_books_count[df_books_count.book_count >= 100]
df_books_count = df_books_count.reset_index()
df_books_count.rename(columns = {'index':'isbn'}, inplace = True)
df_books_count

,isbn,book_count
0,0060392452,158
1,0060502258,166
2,0060915544,117
3,0060928336,320
4,0060930535,218
...,...,...
134,0842329129,155
135,0971880107,581
136,1400034779,183
137,1558743669,134


In [ ]:
# clean books
df_used = df_books_count.merge(df_ratings_new, how='left').drop(['user_count', 'book_count'], 1)
df_used = df_used.dropna(axis = 0, how ='any')
df_used = df_used.reset_index(drop=True)
df_used

,isbn,book_count,user,user_count,rating,title
0,0060392452,158,6251,217,10.0,Stupid White Men ...and Other Sorry Excuses fo...
1,0060392452,158,16634,314,8.0,Stupid White Men ...and Other Sorry Excuses fo...
2,0060392452,158,51883,358,8.0,Stupid White Men ...and Other Sorry Excuses fo...
3,0060392452,158,60244,476,8.0,Stupid White Men ...and Other Sorry Excuses fo...
4,0060392452,158,75591,254,10.0,Stupid White Men ...and Other Sorry Excuses fo...
...,...,...,...,...,...,...
1173,1573229326,104,16795,959,9.0,How to Be Good
1174,1573229326,104,23902,1180,8.0,How to Be Good
1175,1573229326,104,89602,264,5.0,How to Be Good
1176,1573229326,104,110934,225,8.0,How to Be Good


In [ ]:
# add your code here - consider creating a new cell for each section of code

In [ ]:
df_books_features = df_used.pivot(
    index='isbn',
    columns='user',
    values='rating'
).fillna(0)

df_books_features

user,4385,6251,6575,7346,11676,13552,16634,16795,23768,23872,23902,25409,25981,30276,30511,31315,31556,31826,35859,36606,37950,38273,43246,46398,51883,55490,56399,56959,60244,60707,68555,69078,69697,75591,76499,76626,78553,78973,79441,81560,...,162639,168245,171118,172742,174304,177432,177458,182085,185233,189334,189835,190925,197659,200226,204864,216683,217740,225087,225763,230522,234828,235105,236283,240144,242006,248718,249894,252695,254206,254899,257204,258185,261829,262998,264321,265115,265889,269566,270713,274061
isbn,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0060392452,0.0,10.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060502258,0.0,0.0,8.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060915544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060928336,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060930535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0842329129,0.0,0.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0971880107,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1400034779,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0


In [ ]:
# create model and train
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
df_books_features_matrix = csr_matrix(df_books_features.values)
model_knn.fit(df_books_features_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
input = df_books.loc[df['book'] == book]
isbn = input.isbn
 
distances, indices = model_knn.kneighbors(df_books_features.iloc[isbn, :].reshape(1, -1), n_neighbors = 5)

for i in range(len(indices.flatten()):
  recommended_books[i] = df_books_features.index[indices.flatten()[i]]

recommended_books_dist = distances.flatten()

recommended_books = df_books_features.index[indices.flatten()[i]]
recommended_books_dist = df_books_features.index[indices.flatten()[i]], distances.flatten()[i])

  return recommended_books

In [ ]:
query_index = np.random.choice(df_books_features.shape[0]) #return book index
distances, indices = model_knn.kneighbors(df_books_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 5)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(bookByIsbn(df_books_features.index[query_index])))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, bookByIsbn(df_books_features.index[indices.flatten()[i]]), distances.flatten()[i]))

Recommendations for Harry Potter and the Chamber of Secrets (Book 2):

1: Harry Potter and the Goblet of Fire (Book 4), with distance of 0.4085649847984314:
2: Harry Potter and the Prisoner of Azkaban (Book 3), with distance of 0.4613538980484009:
3: The Alienist, with distance of 0.5388268828392029:
4: Where the Heart Is (Oprah's Book Club (Paperback)), with distance of 0.5408631563186646:


In [ ]:
def bookByIsbn(isbn = ''):
  book_title = df_books.title[df_books['isbn'] == isbn].to_list()
  return book_title[0]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()